In [4]:
import pandas
import matplotlib
import seaborn
import lifelines
import sys

sys.path.append('..')

from helpers import FIG_SIZE
from helpers import savefig as _savefig

In [5]:
%matplotlib inline

PALETTE = seaborn.color_palette()
matplotlib.rcParams['figure.figsize'] = FIG_SIZE
savefig = lambda x, y: None # use _savefig to export figures

In [6]:
df_lags = pandas.read_csv('../data/lags.csv.gz', index_col=0)

/home/alexandre/.local/share/virtualenvs/secos-lag/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
df_lags = (
    df_lags
    .assign(
        Time=pandas.to_datetime(df_lags['Time'], infer_datetime_format=True),
        NextTime=pandas.to_datetime(df_lags['NextTime'], infer_datetime_format=True),
        TemporalLagAtTime=pandas.to_timedelta(df_lags['TemporalLagAtTime']),
        TemporalLagAtNextTime=pandas.to_timedelta(df_lags['TemporalLagAtNextTime'])
    )
)

# Overview

## Number of dependencies, releases and packages

In [17]:
df_tmp = pandas.concat([
    # Time
    df_lags['Time'],
    
    # Count dependencies
    (
        df_lags
        .assign(NDependencies=1)
        ['NDependencies']
    ),
    
    # Count releases
    (
        df_lags
        .drop_duplicates(['Package', 'Release'], keep='first')
        .assign(NReleases=1)
        ['NReleases']
    ),
    
    # Count packages
    (
        df_lags
        .drop_duplicates('Package', keep='first')
        .assign(NPackages=1)
        ['NPackages']
    )
], axis=1).fillna({'NReleases': 0, 'NPackages': 0})

In [20]:
df_tmp[['NPackages', 'NReleases', 'NDependencies']].sum()

NPackages         125153.0
NReleases        1643518.0
NDependencies    9730811.0
dtype: float64

In [ ]:
(
    df_tmp
    .set_index('Time')
    .sort_index()
    .cumsum()
    .assign(**{
        'Dependencies': lambda d: d['NDependencies'],
        'Releases x10': lambda d: d['NReleases'] * 10, 
        'Packages x100': lambda d: d['NPackages'] * 100,
    })
    [['Packages x100', 'Releases x10', 'Dependencies']]
    .plot()
)

KeyboardInterrupt: 

# Time lag

## Aggregate time lag for releases

## Time lag distribution

### For dependencies

### For releases

## Time lag at time vs. at next time

### For dependencies

### For releases

## Survival for deps, max rel at both dates, by year

# Version lag

## Evol % deps & rels, missing at least one M, m, p (or percentiles of missed ones?) at date, at next

## Scatter version at date vs next (color for M, m, p)